### Heures de pointe de la STM De 7h à 9h et de 16h à 18h

In [ ]:
from gtfs_functions.gtfs_functions import Feed

folder_gtfs = 'GTFS_files/'

# Definir les tranches horaires
time_windows=[0, 5, 7, 9, 16, 18, 24]

# Permet de selectionner le type de calendrier:
#   - semaine (week)
#   - fin de semaine (weekend) => Resultat errone car les data de samedi et dimanche sont additionnés 
#   - samedi (saturday)
#   - dimanche (dimanche)
#   - Aucun argument => tous les jours sont pris en compte 
#   mais le calcul de la frequence n'est pas juste car les horaires du samedi (par exemple) sont consideres comme un trajet en plus
daysofweek = "week"

# supprime certaines routes dans la liste (ici les lignes de métro)
drop_routes = ["1", "2", "4", "5"]

# supprime les cas de calendrier pour une journée afin d'eviter des double comptage (la suppresssion est la solution simple, a voir pour trouver mieux)
drop_samedate = True

# Ne prend pas en compte les trips associés a un calendar qui commence apres une date
drop_service_after = "20230619"

# Aggrege les routes qui se superpose (cas de la ligne 100 de la STM qui a 2 shapes)
dissolve_routes = True
#### TODO => Aggreger selon les linstring qui se croisent

# aggrege les frequences par stop, pour toutes lignes qui passe a cet arret de bus (stop_freq_by_route doit etre à True)
dissolve_stops = True

# Permet de tagguer les routes associés au stop
stop_freq_by_route= True

feed = Feed(folder_gtfs+"gtfs_stm.zip", 
            time_windows=time_windows,
            daysofweek=daysofweek, 
            busiest_date = False, 
            stop_freq_by_route= stop_freq_by_route,
            drop_routes=drop_routes,
            drop_samedate=drop_samedate,
            drop_service_after= drop_service_after,
            dissolve_route = dissolve_routes,
            dissolve_stops = dissolve_stops)


routes = feed.routes
trips = feed.trips
stops = feed.stops
stop_times = feed.stop_times
shapes = feed.shapes
calendar = feed.calendar

stops_freq = feed.stops_freq
lines_freq = feed.lines_freq

# EXPORTATION DES DONNÉES

In [ ]:
def extract_files_gpkg(df,name ):
    gp = df.groupby('window')
    folder = 'extract_files/frequencies/' + name
    for g in gp.groups:
        file_name = folder + '/' + name +'_frequencies_window_' + str(g) + '.gpkg'
        print(file_name)
        gp.get_group(g).to_file(file_name, driver="GPKG")


#### Extract STOPS FREQUENCIES
extract_files_gpkg(stops_freq,'stops' )

#### Extract LINES FREQUENCIES
extract_files_gpkg(lines_freq,'lines' )

# VALIDATION DES DONNÉES

In [ ]:
display(stops_freq.head(50))
stops_freq.explore()
stop_duplicated = stops_freq.loc[
                stops_freq.duplicated(
                    subset=["stop_id", "direction_id", "window"], keep=False
                ),
                :,
            ].sort_values(by=["stop_id", "direction_id", "window"])
display(stop_duplicated)
# stop_duplicated.explore()


In [ ]:
display(lines_freq)
lines_freq.explore()
lines_duplicated = lines_freq.loc[
                lines_freq.duplicated(
                    subset=["route_id", "direction_id", "window"], keep=False
                ),
                :,
            ].sort_values(by=["route_id", "direction_id", "window"])
display(lines_duplicated)
# lines_duplicated.explore()